In [16]:
!pip install feature_engine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 290 kB 5.2 MB/s 


In [64]:
# The usual suspects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Sklearn
from sklearn.model_selection import (train_test_split, RandomizedSearchCV, 
                                     RepeatedStratifiedKFold, cross_validate)

# Assemble pipeline(s)
from sklearn import set_config
from sklearn.pipeline import make_pipeline, Pipeline
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Handle constant/duplicates and missing features/columns
from feature_engine.selection import (DropFeatures, DropConstantFeatures, 
                                      DropDuplicateFeatures)

# Sampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Models
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance
from scipy.stats import loguniform

set_config(display="diagram")  # make pipeline visible

In [68]:
df = pd.read_csv("/content/Coffee_export.csv")
df.dropna(inplace = True)
X = df.drop('Total_export' , axis = 1)
y = df['Total_export']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
df


,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total_export
0,Angola,5040000,4260000,4800000,2340000,480000,2460000,3120000,3000000,3240000,...,480000,480000,360000,540000,660000,660000,540000,540000,1380000,43320000
1,Bolivia (Plurinational State of),9360000,4440000,5760000,2820000,5040000,5640000,7380000,6660000,5820000,...,4440000,3780000,3300000,3720000,1800000,1320000,1560000,1320000,1200000,137460000
2,Brazil,1016160000,1270980000,1127460000,1070280000,1036380000,868080000,915060000,1008060000,1088640000,...,2028360000,1712940000,1899060000,-2147483648,-2147483648,2056140000,1855500000,2138220000,-2147483648,33807709056
3,Burundi,35100000,41280000,38760000,25080000,30480000,31680000,13440000,31740000,22440000,...,13080000,23520000,11700000,15120000,13800000,12240000,10140000,12120000,17580000,646200000
4,Cameroon,156660000,105120000,98760000,42300000,32760000,24420000,33840000,82080000,44760000,...,29400000,37320000,16320000,22500000,23400000,16860000,14700000,17220000,15000000,1399920000
5,Central African Republic,11820000,8460000,6000000,8220000,8220000,13920000,5880000,12180000,6120000,...,4680000,4680000,60000,4500000,2580000,4800000,1080000,2340000,1140000,170700000
6,Colombia,836640000,755940000,993840000,814080000,706080000,588840000,635280000,655140000,675600000,...,464040000,430200000,580200000,657240000,762960000,769860000,779100000,768480000,820320000,19878480000
7,Congo,120000,60000,0,60000,60000,1140000,780000,180000,0,...,0,0,0,0,0,0,0,0,0,2400000
8,Costa Rica,135960000,144000000,145860000,141840000,126300000,124020000,145800000,125940000,122700000,...,74580000,82440000,80640000,72540000,67680000,60420000,59220000,72480000,62340000,2991840000
9,Côte d'Ivoire,256980000,228300000,272760000,241980000,146640000,149640000,165060000,282780000,261900000,...,46320000,102720000,117720000,89340000,85080000,85920000,51300000,91320000,126240000,4721640000


In [89]:
ppl = Pipeline([
    # Step 1: Drop irrelevant columns/features
   
    ('drop_constant_values', DropConstantFeatures(tol=1, missing_values='ignore')),
    ('drop_duplicates', DropDuplicateFeatures()),
    
    # Step 2: Impute and scale columns/features
    ('cleaning', ColumnTransformer([
        # Step 2.1: Apply steps for numerical features
        ('num',make_pipeline(
            #SimpleImputer(strategy='mean'),
            MinMaxScaler()),
         make_column_selector(dtype_include='int64')
        ),
        # Step 2.2 Apply steps for categorial features
        ('cat',make_pipeline(
            #SimpleImputer(strategy='most_frequent'),
            OneHotEncoder(sparse=False, handle_unknown='ignore')),
         make_column_selector(dtype_include='category')
        )])
    ),
    # Step 3: Sampling
    #('smote', SMOTE()),

        # Step 4: Voting Classifier
    ('ensemble', lr_xgb_rf)

])

In [87]:
ppl_fts = ppl[0:4]
ppl_fts.fit(X_train, y_train)
features = ppl_fts.get_feature_names_out()
pd.Series(features)


0     num__1990
1     num__1991
2     num__1992
3     num__1993
4     num__1994
5     num__1995
6     num__1996
7     num__1997
8     num__1998
9     num__1999
10    num__2000
11    num__2001
12    num__2002
13    num__2003
14    num__2004
15    num__2005
16    num__2006
17    num__2007
18    num__2008
19    num__2009
20    num__2010
21    num__2011
22    num__2012
23    num__2013
24    num__2014
25    num__2015
26    num__2016
27    num__2017
28    num__2018
29    num__2019
dtype: object

In [104]:
# Linear model (logistic regression)
lr = LogisticRegression(penalty = 'l2', solver='saga', warm_start=True, max_iter=400, l1_ratio=1)
# RandomForest
rf = RandomForestClassifier()
# XGB
xgb = XGBClassifier(tree_method="hist", verbosity=0, silent=True)
# Ensemble
lr_xgb_rf = VotingClassifier(estimators=[('lr', lr), ('xgb', xgb), ('rf', rf)], 
                             voting='soft')

In [106]:
# Hyperparameter Tuning
params = {
    'ensemble__lr__solver': ['newton-cg', 'lbfgs', 'liblinear' , 'saga' ],
    'ensemble__lr__penalty': ['none', 'l1', 'l2', 'elasticnet'],
    'ensemble__lr__C': loguniform(1e-5, 100),
    'ensemble__xgb__learning_rate': [0.1],
    'ensemble__xgb__max_depth': [7, 10, 15, 20],
    'ensemble__xgb__min_child_weight': [10, 15, 20, 25],
    'ensemble__xgb__colsample_bytree': [0.8, 0.9, 1],
    'ensemble__xgb__n_estimators': [300, 400, 500, 600],
    'ensemble__xgb__reg_alpha': [0.5, 0.2, 1],
    'ensemble__xgb__reg_lambda': [2, 3, 5],
    'ensemble__xgb__gamma': [1, 2, 3],
    'ensemble__rf__max_depth': [7, 10, 15, 20],
    'ensemble__rf__min_samples_leaf': [1, 2, 4],
    'ensemble__rf__min_samples_split': [2, 5, 10],
    'ensemble__rf__n_estimators': [300, 400, 500, 600],
}

rsf = RepeatedStratifiedKFold(n_splits = 2 , random_state=42)
clf = RandomizedSearchCV(ppl, params,scoring='roc_auc', verbose=2, cv=rsf)
clf.fit(X_train, y_train)

print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)
#print("AUC:", roc_auc_score(y_val, clf.predict(X_val), multi_class="ovr"))

Fitting 20 folds for each of 10 candidates, totalling 200 fits


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/mod

[CV] END ensemble__lr__C=0.0001797779671925815, ensemble__lr__penalty=l1, ensemble__lr__solver=liblinear, ensemble__rf__max_depth=10, ensemble__rf__min_samples_leaf=2, ensemble__rf__min_samples_split=10, ensemble__rf__n_estimators=400, ensemble__xgb__colsample_bytree=1, ensemble__xgb__gamma=2, ensemble__xgb__learning_rate=0.1, ensemble__xgb__max_depth=20, ensemble__xgb__min_child_weight=20, ensemble__xgb__n_estimators=300, ensemble__xgb__reg_alpha=1, ensemble__xgb__reg_lambda=2; total time=   0.8s


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(


[CV] END ensemble__lr__C=0.0001797779671925815, ensemble__lr__penalty=l1, ensemble__lr__solver=liblinear, ensemble__rf__max_depth=10, ensemble__rf__min_samples_leaf=2, ensemble__rf__min_samples_split=10, ensemble__rf__n_estimators=400, ensemble__xgb__colsample_bytree=1, ensemble__xgb__gamma=2, ensemble__xgb__learning_rate=0.1, ensemble__xgb__max_depth=20, ensemble__xgb__min_child_weight=20, ensemble__xgb__n_estimators=300, ensemble__xgb__reg_alpha=1, ensemble__xgb__reg_lambda=2; total time=   0.8s


...

[CV] END ensemble__lr__C=0.003741677293000619, ensemble__lr__penalty=none, ensemble__lr__solver=saga, ensemble__rf__max_depth=15, ensemble__rf__min_samples_leaf=1, ensemble__rf__min_samples_split=5, ensemble__rf__n_estimators=500, ensemble__xgb__colsample_bytree=0.9, ensemble__xgb__gamma=1, ensemble__xgb__learning_rate=0.1, ensemble__xgb__max_depth=20, ensemble__xgb__min_child_weight=20, ensemble__xgb__n_estimators=500, ensemble__xgb__reg_alpha=0.2, ensemble__xgb__reg_lambda=5; total time=   1.1s


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: Convergence

[CV] END ensemble__lr__C=0.003741677293000619, ensemble__lr__penalty=none, ensemble__lr__solver=saga, ensemble__rf__max_depth=15, ensemble__rf__min_samples_leaf=1, ensemble__rf__min_samples_split=5, ensemble__rf__n_estimators=500, ensemble__xgb__colsample_bytree=0.9, ensemble__xgb__gamma=1, ensemble__xgb__learning_rate=0.1, ensemble__xgb__max_depth=20, ensemble__xgb__min_child_weight=20, ensemble__xgb__n_estimators=500, ensemble__xgb__reg_alpha=0.2, ensemble__xgb__reg_lambda=5; total time=   1.1s


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, 

[CV] END ensemble__lr__C=0.003741677293000619, ensemble__lr__penalty=none, ensemble__lr__solver=saga, ensemble__rf__max_depth=15, ensemble__rf__min_samples_leaf=1, ensemble__rf__min_samples_split=5, ensemble__rf__n_estimators=500, ensemble__xgb__colsample_bytree=0.9, ensemble__xgb__gamma=1, ensemble__xgb__learning_rate=0.1, ensemble__xgb__max_depth=20, ensemble__xgb__min_child_weight=20, ensemble__xgb__n_estimators=500, ensemble__xgb__reg_alpha=0.2, ensemble__xgb__reg_lambda=5; total time=   1.0s
Best Score:  nan
Best Params:  {'ensemble__lr__C': 0.0001797779671925815, 'ensemble__lr__penalty': 'l1', 'ensemble__lr__solver': 'liblinear', 'ensemble__rf__max_depth': 10, 'ensemble__rf__min_samples_leaf': 2, 'ensemble__rf__min_samples_split': 10, 'ensemble__rf__n_estimators': 400, 'ensemble__xgb__colsample_bytree': 1, 'ensemble__xgb__gamma': 2, 'ensemble__xgb__learning_rate': 0.1, 'ensemble__xgb__max_depth': 20, 'ensemble__xgb__min_child_weight': 20, 'ensemble__xgb__n_estimators': 300, 'ense